In [ ]:
import pandas as pd
import numpy as np
import clickhouse_connect
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, GRU, Dropout, BatchNormalization, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import os

# Connexion à la base de données ClickHouse
def connect_to_clickhouse():
    return clickhouse_connect.get_client(host='clickhouse', port=8123, username='clickhouse-user', password='secret')

# Chargement des données depuis ClickHouse
def load_data_from_clickhouse(start_date, end_date):
    client = connect_to_clickhouse()
    query = f"""
    SELECT Date, Speed, Density, Bt, Bz, Mag
    FROM solar_data
    WHERE Date BETWEEN '{start_date}' AND '{end_date}'
    ORDER BY Date
    """
    df = client.query_df(query)
    df.dropna(inplace=True)
    return df

# Enrichissement des données avec variables astronomiques
def enrich_data(df):
    df['Date'] = pd.to_datetime(df['Date'])
    df['DayOfYear'] = df['Date'].dt.dayofyear
    df['YearAngle'] = 2 * np.pi * (df['DayOfYear'] - 1) / 365.25
    spring_equinox_day = 80
    df['SeasonAngle'] = 2 * np.pi * ((df['DayOfYear'] - spring_equinox_day) % 365.25) / 365.25
    earth_tilt = 23.44 * np.pi / 180
    df['MagneticTilt'] = earth_tilt * np.cos(df['SeasonAngle'])
    df['MagneticOrientation'] = np.sin(df['SeasonAngle'])
    df['sin_day'] = np.sin(df['YearAngle'])
    df['cos_day'] = np.cos(df['YearAngle'])
    df['sin_hour'] = np.sin(2 * np.pi * df['Date'].dt.hour / 24)
    df['cos_hour'] = np.cos(2 * np.pi * df['Date'].dt.hour / 24)
    df['month'] = df['Date'].dt.month
    df['day'] = df['Date'].dt.day
    df['weekday'] = df['Date'].dt.weekday
    df['hour'] = df['Date'].dt.hour
    df['is_weekend'] = (df['weekday'] >= 5).astype(int)
    df['Speed_to_Density'] = df['Speed'] / (df['Density'] + 1e-6)
    df['Bz_Bt_ratio'] = df['Bz'] / (df['Bt'] + 1e-6)
    return df

# Préparation des données pour l'apprentissage
def prepare_data_for_model(df, sequence_length=24):
    features = ['Speed', 'Density', 'Bt', 'Bz', 'sin_day', 'cos_day', 'MagneticTilt',
                'MagneticOrientation', 'sin_hour', 'cos_hour', 'month', 'day', 'weekday',
                'hour', 'is_weekend', 'Speed_to_Density', 'Bz_Bt_ratio']
    target = 'Mag'
    scaler_X = StandardScaler()
    scaler_y = StandardScaler()
    X_scaled = scaler_X.fit_transform(df[features])
    y_scaled = scaler_y.fit_transform(df[[target]])
    X, y = [], []
    for i in range(len(X_scaled) - sequence_length):
        X.append(X_scaled[i:i + sequence_length])
        y.append(y_scaled[i + sequence_length])
    X = np.array(X)
    y = np.array(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test, scaler_X, scaler_y, features, target

# Création du modèle Bidirectional GRU
def create_model(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        Bidirectional(GRU(64, return_sequences=True)),
        Dropout(0.3),
        Bidirectional(GRU(32)),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

# Entraînement du modèle avec callbacks optimisés
def train_model(model, X_train, y_train, X_test, y_test, epochs=50):
    log_dir = os.path.join("logs", datetime.now().strftime("%Y%m%d-%H%M%S"))
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6),
        ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss', mode='min'),
        TensorBoard(log_dir=log_dir)
    ]
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=32,
                        validation_data=(X_test, y_test), callbacks=callbacks, verbose=1)
    return model, history

# Évaluation du modèle
def evaluate_model(model, X_test, y_test, scaler_y):
    y_pred = model.predict(X_test)
    y_test_inv = scaler_y.inverse_transform(y_test)
    y_pred_inv = scaler_y.inverse_transform(y_pred)
    mse = np.mean((y_test_inv - y_pred_inv) ** 2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(y_test_inv - y_pred_inv))
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")
    plt.figure(figsize=(15, 6))
    sns.set(style="whitegrid")
    plt.plot(y_test_inv[:500], label='Réel', color='blue')
    plt.plot(y_pred_inv[:500], label='Prédit', color='red')
    plt.title('Prédiction des perturbations magnétiques', fontsize=16)
    plt.xlabel('Échantillons', fontsize=14)
    plt.ylabel('Indice de perturbation magnétique', fontsize=14)
    plt.legend(fontsize=14)
    plt.show()
    return mse, rmse, mae, y_test_inv, y_pred_inv

# Fonction de prédiction en temps réel (inchangée ici)
def predict_realtime(model, latest_data, scaler_X, scaler_y, features, sequence_length=24):
    if len(latest_data) < sequence_length:
        raise ValueError("Pas assez de données pour la prédiction en temps réel.")
    latest_data_scaled = scaler_X.transform(latest_data[features])
    latest_sequence = np.array([latest_data_scaled[-sequence_length:]])
    y_pred = model.predict(latest_sequence)
    pred_value = scaler_y.inverse_transform(y_pred)[0, 0]
    kp_scale = classify_magnetic_storm(pred_value)
    latitude = estimate_aurora_latitude(pred_value)
    return {
        'predicted_intensity': pred_value,
        'kp_scale': kp_scale,
        'probable_latitude': latitude,
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }

# Classification & latitude (inchangées)
def classify_magnetic_storm(mag_value):
    if mag_value < 5:
        return "G0 - Faible (Kp < 5)"
    elif mag_value < 6:
        return "G1 - Mineure (Kp = 5)"
    elif mag_value < 7:
        return "G2 - Modérée (Kp = 6)"
    elif mag_value < 8:
        return "G3 - Forte (Kp = 7)"
    elif mag_value < 9:
        return "G4 - Sévère (Kp = 8)"
    else:
        return "G5 - Extrême (Kp = 9)"

def estimate_aurora_latitude(mag_value):
    base_latitude = 65
    latitude_shift = min(20, max(0, (mag_value - 3) * 3))
    return base_latitude - latitude_shift

# Fonction principale

def main_with_user_input():
    start_date = input("Entrez la date de début (YYYY-MM-DD): ")
    end_date = input("Entrez la date de fin (YYYY-MM-DD): ")
    epochs = int(input("Entrez le nombre d'epochs pour l'entraînement: "))

    print("Chargement des données...")
    df = load_data_from_clickhouse(start_date, end_date)

    print("Enrichissement des données...")
    df = enrich_data(df)

    print("Préparation des données pour le modèle...")
    try:
        X_train, X_test, y_train, y_test, scaler_X, scaler_y, features, target = prepare_data_for_model(df)
    except ValueError as e:
        print(e)
        return

    print("Création du modèle...")
    model = create_model((X_train.shape[1], X_train.shape[2]))
    model.summary()

    print("Entraînement du modèle...")
    model, history = train_model(model, X_train, y_train, X_test, y_test, epochs=epochs)

    print("Évaluation du modèle...")
    evaluate_model(model, X_test, y_test, scaler_y)

    print("Sauvegarde du modèle...")
    model.save('solar_magnetic_prediction_model.keras')

    print("Processus terminé!")

    selected_date = input("Entrez une date pour la prédiction (YYYY-MM-DD): ")
    latest_data = df[df['Date'] <= selected_date].tail(24)
    if len(latest_data) < 24:
        print("Pas assez de données historiques pour effectuer une prédiction.")
        return

    prediction_result = predict_realtime(model, latest_data, scaler_X, scaler_y, features)
    print(f"Prédiction pour la date {selected_date}:", prediction_result)

    plt.figure(figsize=(12, 6))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

if __name__ == "__main__":
    main_with_user_input()


2025-04-04 07:59:09.546731: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-04 07:59:09.948477: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-04 07:59:09.959839: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-04 07:59:13.721113: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Entrez la date de début (YYYY-MM-DD):  2017-01-01
Entrez la date de fin (YYYY-MM-DD):  2017-02-01
Entrez le nombre d'epochs pour l'entraînement:  20


Chargement des données...
Enrichissement des données...
Préparation des données pour le modèle...
